In [1]:
import tensorflow as tf
from res_map import responses
res = responses()
import numpy as np

In [2]:
w_conv1 = tf.Variable(tf.truncated_normal([5,5,1,32],stddev=0.1), tf.float32, name = "w_conv1")
b_conv1 = tf.Variable(tf.constant(0.1, tf.float32, shape=[32]), name = "b_conv1")

w_conv2 = tf.Variable(tf.truncated_normal([5,5,32,64], stddev=0.1), tf.float32, name = "w_conv2")
b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]), tf.float32, name="b_conv2")

w_fc1 = tf.Variable(tf.truncated_normal([13*5*64,1024], stddev=0.1), tf.float32, name="w_fc1")
b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]), tf.float32, name = "b_fc1")

w_fc2 = tf.Variable(tf.truncated_normal([1024,282], stddev=0.1), tf.float32, name = "w_fc2")
b_fc2 = tf.Variable(tf.constant(0.1, shape=[282]), tf.float32, name = "b_fc1")

sess = tf.InteractiveSession()
saver = tf.train.Saver()
saver.restore(sess,"data/model_hiv.ckpt")
print "model restored"

model restored


In [3]:
x = tf.placeholder(tf.float32, shape=[None,50,20,1])

convolve1 = tf.nn.conv2d(x,w_conv1,strides=[1,1,1,1], padding="SAME") + b_conv1
h_conv1 = tf.nn.relu(convolve1)
max_pool1 = tf.nn.max_pool(h_conv1, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

layer1 = max_pool1
print "layer1: ",layer1

convolve2 = tf.nn.conv2d(layer1,w_conv2, strides=[1,1,1,1], padding="SAME") + b_conv2
relu2 = tf.nn.relu(convolve2)
max_pool2 = tf.nn.max_pool(relu2, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")
layer2 = max_pool2
print "layer2: ",layer2
layer2_mat = tf.reshape(layer2,[-1,13*5*64])

fully_1 = tf.matmul(layer2_mat, w_fc1) + b_fc1
relu3 = tf.nn.relu(fully_1)
layer3 = relu3
print "layer3: " ,layer3

fully_2 = tf.matmul(layer3, w_fc2) + b_fc2
softy = tf.nn.softmax(fully_2)
layer4 = softy
print "layer4: ",layer4

layer1:  Tensor("MaxPool:0", shape=(?, 25, 10, 32), dtype=float32)
layer2:  Tensor("MaxPool_1:0", shape=(?, 13, 5, 64), dtype=float32)
layer3:  Tensor("Relu_2:0", shape=(?, 1024), dtype=float32)
layer4:  Tensor("Softmax:0", shape=(?, 282), dtype=float32)


In [4]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = [float(val) for val in splitLine[1:]]
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

In [5]:
model = loadGloveModel('glove.6B.50d.txt') # loading word embeddings

Loading Glove Model
Done. 400001  words loaded!


In [31]:
line = "hello"       

checker = np.zeros((50,20))
line = line.lower()
#print line
line_split = line.split()
#print line_split
#wordvecs = []
i = 0;
for word in line_split:
    try:
        checker[:,i] = model[word]
        #wordvecs.append(model[word])
        i+=1    
    except:
        checker[:,i] = model['<unk>']
        #wordvecs.append(model[word])
        i+=1 
check = np.reshape(checker,[-1,50,20,1])        
question_class = np.argmax(sess.run(layer4, feed_dict={x:check}),1)
#print question_class
print res[question_class[0]]

Hii :) 
Wassup 
How may I help you? 

